In [503]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import numpy as np
from loss_model import rmsle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [504]:
data = pd.read_csv('../data/Train.csv')
data

/home/nathan/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,11/2/2011 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,11/2/2011 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,11/2/2011 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,10/25/2011 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [505]:
# Get just the columns we want to look at
data = data.iloc[:, [0,1, 6, 7, 15, 17, 18]]


In [506]:
# Clean YearsMade column
year_mean = data['YearMade'][data['YearMade'] > 1960].mean()
data['YearMade'][data['YearMade'] < 1918] = year_mean

<ipython-input-506-adf5a6ee608a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['YearMade'][data['YearMade'] < 1918] = year_mean
/home/nathan/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [507]:
# Clean ProductSize column
data['ProductSize'] = data['ProductSize'].fillna('Medium') 


<ipython-input-507-f32a02f110a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProductSize'] = data['ProductSize'].fillna('Medium')


In [508]:
# Clean MachinHoursCurrentMeter column
mach_hours_mean = data['MachineHoursCurrentMeter'][data['MachineHoursCurrentMeter'].notnull()].mean()
data['MachineHoursCurrentMeter'] = data['MachineHoursCurrentMeter'].fillna(mach_hours_mean) 

<ipython-input-508-7c53b01cabc1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MachineHoursCurrentMeter'] = data['MachineHoursCurrentMeter'].fillna(mach_hours_mean)


In [509]:
#Create Dummies dataframes
prod_group_dummies = pd.get_dummies(data["ProductGroup"], drop_first=True)
prod_size_dummies = pd.get_dummies(data["ProductSize"], drop_first=True)
state_dummies = pd.get_dummies(data["state"], drop_first=True)
data = pd.concat([data, prod_group_dummies], axis=1)
data = pd.concat([data, prod_size_dummies], axis=1)
data = pd.concat([data, state_dummies], axis=1)
data = data.drop(columns=['ProductGroup', 'ProductSize', 'state'])

In [510]:
X = data.drop(columns=['SalePrice', 'SalesID'] , axis=1)
y = data['SalePrice']
X_scaled = pd.DataFrame(StandardScaler(with_mean=0, with_std=1).fit_transform(X.iloc[:,[0,1]]), columns=['YearMade', 'MachineHoursCurrentMeter'])
X['YearMade'] = X_scaled['YearMade']
X['MachineHoursCurrentMeter'] = X_scaled['MachineHoursCurrentMeter']
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [511]:
X.shape

(401125, 64)

In [542]:
# Linear Regression Model
kf = KFold()

for train_idx, test_idx in kf.split(X_train):
    model = LinearRegression()

    X_train_fold = X_train.iloc[train_idx]
    X_test_fold = X_train.iloc[test_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_test_fold = y_train.iloc[test_idx]
    model.fit(X_train_fold,y_train_fold)
    
    y_hat = model.predict(X_test_fold)
    print(rmsle(y_test_fold, y_hat))
    
print(y_hat)
print(y_test_fold)

/home/nathan/Desktop/dsi_galvanize/case_studies/predicting-heavy-equipment-price/notebooks/loss_model.py:6: RuntimeWarning: invalid value encountered in log
  log_diff = np.log(predictions+1) - np.log(actual+1)


0.5624830152369393


/home/nathan/Desktop/dsi_galvanize/case_studies/predicting-heavy-equipment-price/notebooks/loss_model.py:6: RuntimeWarning: invalid value encountered in log
  log_diff = np.log(predictions+1) - np.log(actual+1)


0.5558271939366481


/home/nathan/Desktop/dsi_galvanize/case_studies/predicting-heavy-equipment-price/notebooks/loss_model.py:6: RuntimeWarning: invalid value encountered in log
  log_diff = np.log(predictions+1) - np.log(actual+1)


0.5527205645419988


/home/nathan/Desktop/dsi_galvanize/case_studies/predicting-heavy-equipment-price/notebooks/loss_model.py:6: RuntimeWarning: invalid value encountered in log
  log_diff = np.log(predictions+1) - np.log(actual+1)


0.5571578212665141
0.5589465142615393
[43114.15007043  7388.40370779 41234.786338   ... 28569.35861404
 23908.43272026  9556.99236703]
304411    20000
95915     12500
7148      65000
358008    17000
169321    21000
          ...  
168967    13000
23469      8250
51069     23500
382699    17500
24977     12750
Name: SalePrice, Length: 60168, dtype: int64


/home/nathan/Desktop/dsi_galvanize/case_studies/predicting-heavy-equipment-price/notebooks/loss_model.py:6: RuntimeWarning: invalid value encountered in log
  log_diff = np.log(predictions+1) - np.log(actual+1)


In [513]:
# Lasso
#model = Lasso()
#model.fit(X_train,y_train)
#y_hat = model.predict(X_test)
#rmsle(y_test, y_hat)

In [514]:
# Ridge
#model = Ridge()
#model.fit(X_train,y_train)
#y_hat = model.predict(X_test)
#rmsle(y_test, y_hat)


In [515]:
X = X.drop(columns=['Alaska', 'Hawaii','Unspecified', 'Vermont', 'Washington DC'])

In [516]:
final_model = LinearRegression()
final_model.fit(X,y)

LinearRegression()

In [517]:
data_test = pd.read_csv('../data/test.csv')
data_test.shape

(11573, 52)

In [518]:
# Get just the columns we want to look at
data_test = data_test.iloc[:, [0, 5, 6, 14, 16, 17]]
data_test

,SalesID,YearMade,MachineHoursCurrentMeter,ProductSize,state,ProductGroup
0,1222837,1000,0.0,Large / Medium,Kentucky,TEX
1,1222839,2006,4412.0,Large / Medium,Connecticut,TEX
2,1222841,2000,10127.0,Large / Medium,Connecticut,TEX
3,1222843,1000,4682.0,Large / Medium,Connecticut,TEX
4,1222845,2002,8150.0,NaN,Florida,WL
...,...,...,...,...,...,...
11568,6333344,2005,NaN,Mini,Texas,TEX
11569,6333345,2005,NaN,Mini,Florida,TEX
11570,6333347,2005,NaN,Mini,Florida,TEX
11571,6333348,2006,NaN,Mini,Texas,TEX


In [519]:
# Clean YearsMade column
year_mean = data_test['YearMade'][data_test['YearMade'] > 1960].mean()
data_test['YearMade'][data_test['YearMade'] < 1918] = year_mean

In [520]:
# Clean ProductSize column
data_test['ProductSize'] = data_test['ProductSize'].fillna('Medium') 

In [521]:
# Clean MachinHoursCurrentMeter column
mach_hours_mean = data_test['MachineHoursCurrentMeter'][data['MachineHoursCurrentMeter'].notnull()].mean()
data_test['MachineHoursCurrentMeter'] = data_test['MachineHoursCurrentMeter'].fillna(mach_hours_mean) 

In [522]:
#Create Dummies dataframes
prod_group_dummies = pd.get_dummies(data_test["ProductGroup"], drop_first=True)
prod_size_dummies = pd.get_dummies(data_test["ProductSize"], drop_first=True)
state_dummies = pd.get_dummies(data_test["state"], drop_first=True)
data_test = pd.concat([data_test, prod_group_dummies], axis=1)
data_test = pd.concat([data_test, prod_size_dummies], axis=1)
data_test = pd.concat([data_test, state_dummies], axis=1)
data_test = data_test.drop(columns=['ProductGroup', 'ProductSize', 'state'])

In [523]:
data_test.shape

(11573, 60)

In [524]:
y_actual = pd.read_csv('../data/test_actual.csv')

In [525]:
data_test = data_test.drop('SalesID', axis=1)

In [526]:
for col in X.columns:
    if col not in data_test.columns:
        print(col)

In [527]:
y_hat = final_model.predict(data_test)

In [537]:
data_test

,YearMade,MachineHoursCurrentMeter,MG,SSL,TEX,TTT,WL,Large,Large / Medium,Medium,...,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,1999.506948,0.00000,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2006.000000,4412.00000,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2000.000000,10127.00000,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1999.506948,4682.00000,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2002.000000,8150.00000,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11568,2005.000000,5482.14138,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
11569,2005.000000,5482.14138,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11570,2005.000000,5482.14138,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11571,2006.000000,5482.14138,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [529]:
rmsle(y_actual['SalePrice'], y_hat)

6.589712137390767

array([19069825.19259936, 19012072.68109632, 18786931.701944  , ...,
       18937799.57071299, 18947545.70759673, 18948468.17662074])

In [544]:
y_actual

,SalesID,SalePrice,Usage
0,1222837,31000.0,PublicTest
1,1222839,54000.0,PublicTest
2,1222841,26500.0,PublicTest
3,1222843,10000.0,PublicTest
4,1222845,35000.0,PublicTest
...,...,...,...
11568,6333344,10000.0,PublicTest
11569,6333345,10500.0,PublicTest
11570,6333347,12500.0,PublicTest
11571,6333348,10000.0,PublicTest
